# CMPT 713 Project: Explore Prototype of Enterprise Domain Question & Answer

## 1 Motivation
Around 70% of enterprise internal IT, Finance, and HR inquiries are basic and repetitive, requiring significant human effort and resulting in high response times. AI-powered QA tools offer a promising solution to address these challenges by reducing workloads, improving response times, and enhancing collaboration efficiency. Internal inquiries often involve enterprise-specific and sensitive information, making the use of public tools like Chat-GPT a security risk. A privately owned, internal AI tool can balance operational efficiency with the need for strict information security, highlighting a billion-dollar market opportunity.

 Our project focused on developing an AI tool prototype for internal IT inquiries and answers, with the potential to expand to HR and Finance domains by adapting datasets.

## 2 Approach

### 2.1 Related Work
The IBM TechQA dataset[1], designed for domain adaptation, provides a real-world corpus of questions derived from technical forums and answers linked to IBM Technotes. The dataset is quite technical and not common questions and answers we encounter in peoples' daily life. TECHQA questions and answers are substantially longer than those found in common datasets, which will be described in detail in data part. [1] also provided some QA baseline using pre-trained BERT-base model or finetuned verion based on them. 

Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks[2] presents a model that combines retrieval and generation, which improves the model’s ability to handle knowledge-intensive tasks. It uses a retrieval system to fetch relevant information from external sources (e.g., Wikipedia) and then feeds that into a generative model to produce more accurate and informative answers. The model outperforms traditional ones, especially in tasks like open-domain and abstractive question answering. Two methods, RAG-Token and RAG-Sequence, differ in how they retrieve and use documents during generation.

Enterprise Knowledge Retrival Generation(EKRG)[3] introduces a retrieval-generation framework for enterprise knowledge bases. EKRG employs instruction-tuned large language models (LLMs) to generate pseudo question-document pairs, addressing the scarcity of labeled data. The retrieval component is enhanced using a relevance-aware teacher-student strategy, while the generation module employs chain-of-thought (CoT) fine-tuning to mitigate LLM hallucination and improve reasoning. 

TechQA is a dataset that could resemble an enterprise IT knowledge base. Given that Retrieval-Augmented Generation (RAG) is well-suited for handling knowledge-intensive tasks, building our prototype based on the RAG approach using TechQA as the dataset appears promising.

### 2.2 Our Approach - RAG
Our approach utilizes a subset of the TechQA dataset, focusing on a narrower scope of answerable questions within the training and test datasets of IBM TechQA. We refer to this refined subset as the Relevant Tech Notes Dataset (RTND). While the overall pipeline is inspired by the core design of Retrieval-Augmented Generation (RAG), we have implemented key modifications to improve its effectiveness.

#### 2.2.1 First Part of the Pipeline - Retrieval
The RTND dataset is divided into manageable chunks, and each chunk is embedded using the “all-mpnet-base-v2” model to generate dense vector representations, while the query is embedded into the same vector space for similarity-based retrieval. Using these embeddings, the top K most relevant chunks are retrieved based on their similarity scores with the query embedding, ensuring the scores exceed a pre-defined threshold for quality assurance. To provide complete context, the source tech notes corresponding to the selected chunks are then retrieved, ensuring that all necessary information from the relevant documents is available for the subsequent stages of the pipeline to operate effectively.

In the first part, we conduct experiments using BM25, a term-based ranking model, alongside sentence embeddings to incorporate semantic understanding. The following sections detail the algorithms and models employed, emphasizing their connections to prior research.

##### 2.2.1.1 BM25 for Term-Based Retrieval

BM25 is a classic ranking function introduced in the context of probabilistic IR models[4]. It scores documents based on query term frequency while normalizing for document length. The formula is:

$
\text{BM25}(q, d) = \sum_{t \in q} \text{IDF}(t) \cdot \frac{f(t, d) \cdot (k_1 + 1)}{f(t, d) + k_1 \cdot \left(1 - b + b \cdot \frac{|d|}{\text{avgdl}}\right)}
$

- $q$, $d$: Query and document.
- $f(t, d)$: Term frequency.
- $k_1, b$: Hyperparameters.

Because the model is simple and effective, it has been widely adopted by academic and industrial search systems. In our project, we take it as a poential replace fir the embedding model in the RAG architecture with BM25 for the retrieval stage to find relevant documents.


#####  2.2.1.2 Sentence Embeddings for Semantic Understanding

To capture deeper semantic relations, we employ pre-trained sentence transformers[5]. These models map text to a dense vector space where semantically similar texts have close embeddings. The cosine similarity between query ($\mathbf{v}_q$) and document ($\mathbf{v}_d$) embeddings is computed as:

$
\text{Cosine Similarity}(\mathbf{v}_q, \mathbf{v}_d) = \frac{\mathbf{v}_q \cdot \mathbf{v}_d}{\|\mathbf{v}_q\| \|\mathbf{v}_d\|}
$

Our approach enhances prior work by aggregating chunk embeddings using methods like mean pooling, inspired by work on passage-level retrieval[6].

#### 2.2.2 Second Part of the Pipeline - Generation
The source tech notes associated with the top K chunks are utilized as the context for generating answers in the subsequent stages of the pipeline. To optimize this process, we experimented with different models, exploring various approaches to effectively leverage the retrieved context and improve the accuracy and relevance of the generated answers.

 ##### 2.2.2.1 Roberta-v3:
 As the first approach, we utilized the Roberta-v3 model to generate answers based on the retrieved context. To enable accurate extractive question answering, we fine-tune a pre-trained transformer model, `microsoft/deberta-v3-large`, on a dataset of question-context-answer triples. The fine-tuning process involves predicting the start and end positions of the answer within the given context. The model generates probability distributions over all possible positions for the start and end of the answer, using the hidden states of the transformer and two learnable projection layers. The training objective minimizes the cross-entropy loss for both start and end positions:
$\mathcal{L} = \frac{1}{2} \left( \mathcal{L}_{\text{start}} + \mathcal{L}_{\text{end}} \right)$ where$\mathcal{L}_{\text{start}}$ and $\mathcal{L}_{\text{end}}$ represent the cross-entropy losses for the predicted and true start and end positions, respectively. To handle long contexts and ensure scalability, the model uses tokenized inputs truncated to a fixed maximum length. Special handling is implemented for cases where no answer exists in the context, assigning a default prediction: `I don't have enough information to answer that.`.

 ##### 2.2.2.2 Llama2-7B:
 In this approach, we first embed both the query and the retrieved context using the BAAI/bge-large-en model. These embeddings are stored and queried using Chroma as the vector database. The retrieved context, together with the query, is then passed to the meta-llama/Llama-2-7b-chat-hf model, which generates the final answer.

 ##### 2.2.2.3 LLM API:
 In this approach, we passed the retrieved notes from the first part of the pipeline as context, together with the question to [Deepseek API](https://api-docs.deepseek.com/), using model *deepseek-chat* to generate the final answers. In order to get comparable results with extration approach, we did Prompt Engieering to ask the LLM to only return context without providing any inference.

   
## 3 Data
We utilize a subset of TechQA as our data. The RTND (relevant tech notes dataset) is refering to the `training_dev_technotes.json`. The training dataset we use is the `training_Q_A.json`. The dev dataset we use is the `dev_Q_A.json`. The files could be found in traing_and_dev folder under the `TechQA.tar.gz` from https://huggingface.co/datasets/PrimeQA/TechQA/tree/main

The key difference of TechQA compared to SQuAD 2.0 and HOTPOTQA lies in the length and complexity of its questions and answers. TechQA features significantly longer questions (mean ~52 tokens) and answers (mean ~48 tokens), whereas SQuAD 2.0 and HOTPOTQA have much shorter questions (mean ~10 tokens for SQuAD and ~17 tokens HOTPOTQA) and answers (mean ~3 tokens for SQuAD 2.0 and ~2 tokens for HOTPOTQA). Additionally, TechQA exhibits higher variability in lengths (greater standard deviation), reflecting more complex and detailed queries, making it more challenging than the other two datasets.

## 4 Code

### 4.1 Retrieval
- `retrieve.py`: Provides retrieval functions, including methods such as index building (`build_faiss_index`) and retrieval (`retrieve_documents_with_scores_new`) based on Faiss. The core logic is custom implemented by the project, but some functions (such as Faiss) are based on open source libraries.
- `embedding.py`: Generate document embeddings, including long text processing, embedding aggregation and other functions, supporting batch processing and progress display. The core embedding generation method may be based on third-party libraries (such as `sentence-transformers`), but long text processing and aggregation are custom implementations of the team.
- `bm25.py`: Implement BM25 preprocessing, index building and retrieval functions based on the `rank_bm25` open source library.The main functions rely on the open source library `rank_bm25`, but document preprocessing (such as word segmentation and stop word filtering) is implemented by the team.


### 4.2 Generation
- `llm_api.py`: perform answer generation with Deepseek API approach.
- `fine_tune.py`: Provide model fine-tuning process, load datasets for question-answering tasks, train models and save them. The fine-tuning logic is implemented by the team and relies on open source libraries (such as Transformers) to complete training.
- `llama2-gen.py`: perform answer generation with llama2, the code is developed referring to open tutorial in [7]

### 4.3 Experiment
- `run_experiment.py`: Provides the main program entry, responsible for parameter parsing, path initialization, and calling core modules (such as BM25, embedding generation, retrieval, etc.) to complete document processing and question answering processes.

### 4.4 Evaluation
The evaluation process follows the framework of the homework codes.
- `check.py`: realize overall evaluations to retrieval and generation outputs, using measures defined in `recall.py` and `bleu.py`; borrowed from homework code with adaption.
- `recall.py`: calculate measures of Recall and rRecall
- `bleu.py`: calculate BLEU core, borrowed from homework code with adaption.
- `iocollect.py`: support code for `check.py`; borrowed from homework code.
- `zipout.py`: only compress output folder to `output.zip`.

### 4.5 Others
- `util.py`: Provides common tool functions, such as document loading, segmentation, preprocessing, etc., and supports multiple module calls. Customized by the team, without direct reference to job code or external libraries.

## 5 Experimental Setup
### 5.1 Retrieval Part
In this part, the objective is find the target note from a retrieval set with the smallest k. 

Besides the general Recall@k, we estabilished a refined rRecall to give punishment on bigger returned note ID set.

When the returned ID set covers the target ID, it's called *hit*.

|         | Per Question                      | Overall                                                | Answerable                                       | Non-Answerable                                |
|---------|-----------------------------------|--------------------------------------------------------|--------------------------------------------------|-----------------------------------------------|
| Recall  | $$hit = 1$$                       | $$RecallScore = \frac{sum_{hit}}{total_{Question}}$$   | $$ABRecall = \frac{sum_{ab-hit}}{total_{ab}}$$   | $$NARecall= \frac{sum_{na-hit}}{total_{na}}$$ |
| rRecall | $$rhit = \frac{1}{length_{set}}$$ | $$rRecallScore = \frac{sum_{rhit}}{total_{Question}}$$ | $$rABRecall = \frac{sum_{ab-rhit}}{total_{ab}}$$ | same as $NARecall$                            |

We evaluated model performance using both Recall and rRecall (as defined above). However, we prioritized **rRecall** to select the best model for this step, as it aligns with our focus on retrieval performance evaluation.

### 5.2 Generation Part
According to [the TechQA Dataset](https://arxiv.org/pdf/1911.02984), F1 was used to evaluation the model performance of answer extraction. However, our project included LLM generation methods. Therefore, we chose **BLEU score** as the measure to compare the outputs from different models.

We used `sacrebleu.metrics.BLEU(effective_order=True).sentence_score()` to calculate the BLEU score for every answer single, withthe `effective_order=True` option to includ n-gram precision for each n-gram order (up to 4-grams typically) and the proper handling of the brevity penalty.

$$
\text{BLEU}(c) = BP \cdot \exp \left( \frac{1}{N} \sum_{n=1}^{N} \log p_n \right)
$$

Where:
- $p_n$ is the precision for n-grams.
- $N$ is the maximum n-gram length (usually 4).
- $BP$ is the brevity penalty.

## 6 Results
### 6.1 Retrieval

| Methods             | Train – rABRecall(%) | Dev – rABRecall(%) |
|---------------------|----------------------|--------------------|
| bm25-top10          | 3.6                  | 3.6                |
| bm25-top20          | 2.3                  | 2.3                |
| embed-top10-0.6     | 13.0                 | 13.1               |
| embed-top20-0.6     | 12.0                 | 12.2               |
| embed-top30-0.6     | 11.6                 | 11.9               |
| **embed-top10-0.7** | **14.8**             | **13.6**           |
| embed-top20-0.7     | 14.5                 | 13.4               |
| embed-top30-0.7     | 14.5                 | 13.4               |

### 6.2 Generation
Comparison based on question and the target note as input for answer outputs.
BLEU was calculated on answerable questions.

| Model            | BLEU - train | BLEU - dev  | Avg words - train | Avg words - dev |
|------------------|--------------|-------------|-------------------|-----------------|
| ReberTa-v3       | 1.532        | 1.0964      | 22.8              | 19.7            |
| ReberTa-v3 + FT  | 1.1885       | 0.9853      | 12.2              | 14.8            |
| Llama            | 8.1798       | 11.5666     | 91.9              | 70.6            |
| **DeepSeek API** | **31.0984**  | **28.5938** | 60.6              | 64.3            |
| *Ground Truth*   |              |             | *48.9*            | *41.9*          |


### 6.3 Selected Pipeline
- Retrieval: Embedding (k=2, threshold=0.7)
- Generation: Deepseek API; if no note was retrieved from Retrieval, skip the answer generation and return a blank string.

BLEU was calculated on questions with generated answers.
 
|            | RecallScore - train | RecallScore - dev | BLEU - train | BLEU - dev |
|------------|---------------------|-------------------|--------------|------------|
| Overall    | 29.0000             | 36.1290           |              |            |
| Answerable | 25.1111             | 23.1250           | 19.3898      | 11.7768    |

## 7 Analysis of the Results
### 7.1 Retrieval
- Baseline (embed-top10-0.6):
  - Recall scores of **13.0% (train)** and **13.1% (dev)** served as the initial baseline.
  - This configuration used embedding-based retrieval but applied a lower cosine similarity threshold 0.6 and retrieved less candidates k=10. While effective, it suffered from slower performance due to the small number of retrieved candidates and less stringent filtering.

- Transition to BM25:
  - To address the speed issue of embedding retrieval, BM25 was explored as a potential alternative. However, as shown in the results, BM25’s recall scores 3.6% were significantly lower than the baseline, even when retrieving up to \(k=20\).
  - This poor performance highlights BM25’s limitation in semantic retrieval tasks, where term-based matching fails to capture the meaning of queries effectively.

- Improved Embedding Retrieval:
  - The refined embedding-based approach (**embed-top10-0.7**) achieves the best recall scores:
    - Train: **14.8%**
    - Dev: **13.6%**
  - By increasing the similarity threshold 0.7, this configuration strikes a balance between speed and precision. The smaller result set reduces computational overhead, and the higher threshold ensures higher-quality results.

**Conclusion**:
- BM25 was discarded due to poor recall performance despite its speed advantage.
- The optimized embedding retrieval (**embed-top10-0.7**) outperformed the baseline, addressing both recall and efficiency concerns.


### 7.2 Generation

- Baseline: ReberTa-v3:
   - ReberTa-v3 served as the baseline for answer generation. Its BLEU scores were extremely low, with **1.532 (train)** and **1.0964 (dev)**, indicating a lack of understanding and precision in generating answers.

- Comparison with Other Models:
    - ReberTa-v3 + FT:
      - The fine-tuned ReberTa-v3 model slightly improved its ability to generate shorter answers, but BLEU scores remained low at **1.1885 (train)** and **0.9853 (dev)**.
      - This highlights the model's limitations, even with task-specific fine-tuning.
    - Llama2-7B:
      - The Llama model achieved a notable improvement, with BLEU scores of **8.18 (train)** and **11.57 (dev)**. However, its generated answers were overly verbose, averaging **91.9 words (train)**. This verbosity made the answers less concise and practical for users.
    - DeepSeek API:
      - DeepSeek API significantly outperformed the baseline, with BLEU scores of **31.1 (train)** and **28.6 (dev)**. It also generated concise answers averaging **60.6 words (train)**, much closer to the ground truth's **48.9 words**.
      - This model demonstrated a strong balance between answer quality and length, making it the best choice for generation.

**Conclusion**:
The DeepSeek API provides substantial improvements over ReberTa-v3 and its fine-tuned variant, achieving BLEU scores that are nearly **30x higher than the baseline**.

### 7.3 Conclusion
- The improved retrieval pipeline feeds high-quality, semantically relevant documents into the generation model, enabling better outputs.
- BLEU scores indicate a substantial improvement in the quality of generated answers compared to the noisy baseline.
#### Why Not Further Improvement?

- Recall Gap: Despite improvements, recall scores (~36%) suggest there’s still room to optimize retrieval, potentially through hybrid methods (e.g., combining BM25 and embeddings).
- BLEU Ceiling: The gap between BLEU and ground truth indicates limitations in handling highly complex or ambiguous questions.


## 8 Future Work
The IBM TECHQA dataset’s technical Q&A content may have been underrepresented during LLM training, potentially impacting model performance in this domain. However, several approaches are worth exploring in the future:
- The current pipeline utilizing the DeepSeek API shows promise for enterprise QA tasks where information security concerns are lower.
- Experimenting with Llama3 could further enhance the pipeline’s performance, making it better suited for technical Q&A tasks.
- The pipeline based on Llama2 may be more appropriate for less complex domains beyond technical Q&A.
- Fine-tuning LLMs specifically for technical Q&A tasks offers a opportunity to improve both the accuracy and relevance of responses in this specialized domain.
- Fine-tuning the ReberTa model could be enhanced by adopting a two-step process: First, leveraging a pre-trained model fine-tuned on SQuAD QA tasks to establish a strong general QA foundation. Second, conducting further fine-tuning using the TECHQA dataset to adapt the model to domain-specific requirements.



## 9 References

[1] Vittorio Castelli and Rishav Chakravarti and Saswati Dana1, The TechQA Dataset, arXiv:1911.02984v1 [cs.CL] 8 Nov 2019

[2] Patrick Lewis and Ethan Perez,Retrieval-Augmented Generation for Knowledge-Intensive NLP Task,arXiv:2005.11401 [cs.CL]

[3] Feihu Jiang and Chuan Qin, Enhancing Question Answering for Enterprise Knowledge Bases using Large Language Models,arXiv:2404.08695v2 [cs.CL] 20 Apr 2024

[4]	S. E. Robertson and K. S. Jones, “Relevance weighting of search terms,” J. Am. Soc. Inf. Sci., vol. 27, no. 3, pp. 129–146, 1976.

[5]	N. Reimers and I. Gurevych, “Sentence-BERT: Sentence embeddings using Siamese BERT-networks,” arXiv [cs.CL], 2019.

[6]	K. Vladimir et al., “Dense passage retrieval for open-domain question answering,” arXiv [cs.CL], 2020.

[7] Gabriel Preda,RAG using Llama 2,Langchain and ChromaDB, https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb